# Librerias

In [1]:
%pip install scrapy
!scrapy startproject wikiscraper
%cd wikiscraper/wikiscraper/spiders/

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


New Scrapy project 'wikiscraper', using template directory 'C:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\scrapy\templates\project', created in:
    C:\Users\Usuario\Documents\University\Recuperacion Web\LibreriaUNAL\Trabajo 2\wikiscraper

You can start your first spider with:
    cd wikiscraper
    scrapy genspider example example.com
c:\Users\Usuario\Documents\University\Recuperacion Web\LibreriaUNAL\Trabajo 2\wikiscraper\wikiscraper\spiders


# Funciones

# Web Scrapping

In [3]:
!scrapy genspider wikiSpider es.wikipedia.org/wiki/Programaci%C3%B3n

C:\Users\Usuario\Documents\University\Recuperacion Web\LibreriaUNAL\Trabajo 2\wikiscraper\wikiscraper\spiders\wikiSpider.py already exists


In [6]:
!scrapy runspider wikiSpider.py -O ../../ejercicio0.json

# # main driver
# if _name_ == "_main_":
    
#     # Run the Spider
#     process = CrawlerProcess()
#     process.crawl(WikiSpider)
#     process.start()

Usage

2023-10-28 17:45:08 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: wikiscraper)
2023-10-28 17:45:08 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Windows-10-10.0.22621-SP0



=====
  scrapy runspider [options] <spider_file>

Run the spider defined in the given file

Options
  -h, --help            show this help message and exit
  -a NAME=VALUE         set spider argument (may be repeated)
  -o FILE, --output FILE
                        append scraped items to the end of FILE (use - for
                        stdout), to define format set a colon at the end of
                        the output URI (i.e. -o FILE:FORMAT)
  -O FILE, --overwrite-output FILE
                        dump scraped items into FILE, overwriting any existing
                        file, to define format set a colon at the end of the
                        output URI (i.e. -O FILE:FORMAT)
  -t FORMAT, --output-format FORMAT
                        format to use for dumping items

Global Options
--------------
  --logfile FILE        log file. if omitted stderr will be used
  -L LEVEL, --loglevel LEVEL
                        log level (default: DEBUG)
  --nolog               disa

# Pre procesamiento

# Vectorizacion

# Base de Datos

# Consulta